In [1]:
import sys
import os
path = os.path.abspath('../..')
if path not in sys.path:
  sys.path.insert(0, path)

%load_ext autoreload

In [3]:
#!pip install sentence_transformers

In [2]:
import collections
import concurrent.futures
import datetime
import json
import termcolor

import numpy as np

from IPython import display

from concordia.agents import basic_agent
from concordia.components import agent as components
from concordia import components as generic_components
from concordia.associative_memory import associative_memory
from concordia.associative_memory import blank_memories
from concordia.associative_memory import formative_memories
from concordia.associative_memory import importance_function
from concordia.clocks import game_clock
from concordia.components import game_master as gm_components
from concordia.environment import game_master
from concordia.metrics import goal_achievement
from concordia.metrics import common_sense_morality
from concordia.metrics import opinion_of_others
from concordia.utils import html as html_lib
from concordia.utils import measurements as measurements_lib
from concordia.utils import plotting

from examples.custom_components import behavior

In [3]:
local_models = True

if local_models:
    # Setup LLM
    from concordia.language_model import ollama_model
    model = ollama_model.OllamaLanguageModel(
        model_name='llama2:70b',
        streaming=True
    )

    # Setup measurements and clock
    measurements = measurements_lib.Measurements()
    time_step = datetime.timedelta(minutes=20)
    SETUP_TIME = datetime.datetime(hour=20, year=2024, month=10, day=1)

    START_TIME = datetime.datetime(hour=18, year=2024, month=10, day=2)
    clock = game_clock.MultiIntervalClock(
        start=SETUP_TIME,
        step_sizes=[time_step, datetime.timedelta(seconds=10)])

    # Setup sentence encoder
    from sentence_transformers import SentenceTransformer
    st5_model = SentenceTransformer('sentence-transformers/sentence-t5-base')
    embedder = st5_model.encode

else:
    from concordia.language_model import gpt_model
    from concordia.language_model import gcloud_model
    # @title Language Model - pick your model and provide keys
    CLOUD_PROJECT_ID = '' #@param {type: 'string'}
    GPT_API_KEY = '' #@param {type: 'string'}
    GPT_MODEL_NAME = '' #@param {type: 'string'}

    USE_CLOUD = True #@param {type: 'boolean'}

    if USE_CLOUD:
        model = gcloud_model.CloudLanguageModel(project_id= CLOUD_PROJECT_ID)
    else:
        model = gpt_model.GptLanguageModel(api_key=GPT_API_KEY, model_name=GPT_MODEL_NAME)

#@title Importance models
importance_model = importance_function.ConstantImportanceModel()
importance_model_gm = importance_function.ConstantImportanceModel()


In [4]:
# @title Generic memories are memories that all players and GM share.

shared_memories = [
    'Alice returns late from work and finds Dorothy at her doorstep.',
    "Dorothy has to move out of her house tonight and needs someone to help her move.",
]

# The generic context will be used for the NPC context. It reflects general
# knowledge and is possessed by all characters.
shared_context = model.sample_text(
    'Summarize the following passage in a concise and insightful fashion:\n'
    + '\n'.join(shared_memories)
    + '\n'
    + 'Summary:'
)
print(shared_context)

/opt/homebrew/Caskroom/miniconda/base/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(



Alice returns home from work to find Dorothy waiting for her, needing assistance with a last-minute move.
Alice returns home from work to find Dorothy waiting for her, needing assistance with a last-minute move.


In [5]:
blank_memory_factory = blank_memories.MemoryFactory(
    model=model,
    embedder=embedder,
    importance=importance_model.importance,
    clock_now=clock.now,
)

formative_memory_factory = formative_memories.FormativeMemoryFactory(
    model=model,
    shared_memories=shared_memories,
    blank_memory_factory_call=blank_memory_factory.make_blank_memory,
)

In [7]:
#@title Creating character backgrounds, goals and traits. Modify to explore how it influences the outcomes
NUM_PLAYERS = 2

scenario_premise = [

    (
        'Alice returns late from work and finds her friend Dorothy at her doorstep. '
        + 'Dorothy has to move out of her house tonight and needs someone to help her move.'
    ),
]
player_configs = [
    formative_memories.AgentConfig(
        name='Alice',
        gender='female',
        goal='Alice is exhausted after work, and she needs to help her grandmother all day the next day.',
        context=shared_context,
        traits='responsibility: medium; aggression: medium',
    ),
    formative_memories.AgentConfig(
        name='Dorothy',
        gender='female',
        goal=(
            "Dorothy has planned on Bob helping her move, but Bob didn't show up. "
        ),
        context=shared_context,
        traits='responsibility: medium; aggression: medium',
    ),
]

In [8]:
mem = formative_memory_factory.make_memories(player_configs[0])

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Alice is a 30-year-old woman who works as a graphic designer at a marketing firm. She has a medium level of responsibility and aggression. A typical day for Alice involves creating visual content for various clients, meeting with her team to discuss project progress, and collaborating with colleagues to ensure timely delivery of designs.

Alice's primary goal is to become a creative director at her firm. She hopes to excel in her career, take on more significant projects, and mentor junior designers. Additionally, Alice desires to start her own business, selling her artwork online. She dreams of having her designs featured in prominent magazines and galleries.

Alice's duties include attending client meetings, creating design concepts, and managing projects from start to finish. She is responsible for ensuring that all designs are of high quality and meet the clients' needs. Alice also has obligations outside work, such as taking care of her elderly mother and volunteering at a local a

Batches:   0%|          | 0/1 [00:00<?, ?it/s]


At the age of 7, Alice was playing outside with her friends when they decided to have a race. Alice was determined to win and ran as fast as she could, but she tripped and fell, scraping her knee. Her friends helped her up and encouraged her to keep going, but Alice was too embarrassed and felt like quitting. Her mother, who was watching from the window, came outside and reminded Alice that it's okay to make mistakes and that she shouldn't give up. This experience taught Alice the value of perseverance and the importance of learning from her mistakes, which has been beneficial in her graphic design career where she often faces tight deadlines and has to problem-solve quickly.

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]


At the age of 21, Alice was working on a critical design project for a client when she realized that she had made a major mistake in the layout. Feeling stressed and anxious, Alice called her mother, who calmed her down and suggested that she take a break and approach the problem with fresh eyes. With her mother's advice, Alice was able to correct the mistake and deliver the project on time, learning the importance of seeking help when needed and taking breaks to avoid burnout. This experience has been particularly relevant in Alice's graphic design career, where she often faces tight deadlines and has to problem-solve quickly.

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [9]:
%autoreload 2
from examples.custom_components import behavior

behaviors = behavior.BehavioralChoices(
    name='behaviours',
    model=model,
    memory=mem,
    player_config=player_configs[0],
    clock_now=clock.now,
    verbose=False
  )

behaviors.update()

1. Alice could offer to help Dorothy move her belongings into storage.
2. Alice could offer to help Dorothy find a new place to live.
3. Alice could offer to help Dorothy pack up her belongings.
4. Alice could offer to help Dorothy sort through her belongings and decide what to keep, sell or donate.
5. Alice could offer to help Dorothy transport some of her belongings to a new location.
6. Alice could suggest that Dorothy hire professional movers to help with the move.
7. Alice could offer to watch Dorothy's dog while she is moving.
8. Alice could offer to lend Dorothy some packing supplies, such as boxes or bubble wrap.
9. Alice could offer to help Dorothy disassemble and reassemble furniture for the move.
10. Alice could suggest that Dorothy postpone the move until a later date when Alice is less busy with work.

The likelihood of each behavior will depend on various factors, such as Alice's energy level, available resources, and her relationship with Dorothy. For example, offering t

In [10]:
behaviors._memory.retrieve_recent(k = 100)

["At the age of 3, Alice was playing with her building blocks when she knocked over her tower, causing her to become frustrated and cry. Her mother, who was in the kitchen, heard Alice's cries and came into the living room to comfort her. She helped Alice rebuild the tower, teaching her how to stabilize it so it wouldn't fall again. This experience taught Alice the importance of perseverance and problem-solving skills, which has stayed with her throughout her life and has been particularly relevant in her career as a graphic designer.",
 "\nAt the age of 7, Alice was playing outside with her friends when they decided to have a race. Alice was determined to win and ran as fast as she could, but she tripped and fell, scraping her knee. Her friends helped her up and encouraged her to keep going, but Alice was too embarrassed and felt like quitting. Her mother, who was watching from the window, came outside and reminded Alice that it's okay to make mistakes and that she shouldn't give up. 

In [12]:
attitude_ratings = model.sample_text(
  f'Memories of Alice:{behaviors._memory.retrieve_recent(100)}'
  f'Instructions: \n'
  f'Given the memories above, generate a list of five potential behaviours that alice can take in response to the situation. '
  f'For each potential behaviour, provide three possible positive and three possible negative consequences of that behaviour. '
  f'For each potential consequence, indicate with a number from -10 to 10 how bad to good '
  f'that consequence is for Alice. -10 is the worst possible outcome, 0 is neutral, and 10 is the best possible outcome.'
  f'List the positive or negative value after each of the separate consequences to each behaviour. '
  f'For each potential behaviour, provide just the consequence numbers separated by a comma. '
  f'For each potential consequence, indicate with a number from 0 to 100 how likely that consequence is to occur. 0 is impossible, 100 is certain. '
  f'List the likelihood of the consequences after each of the separate consequences to each behaviour. So, '
  f'each of the 6 potential separate consequences should have a likelihood value. '
  f'This should be in the form of (Value: number, Likelihood: number) for each potential consequence, '
  f'Remember, there should be three separate positive and three negative consequences for each potential behaviour '
  f'each with its own value and likelihood.\n'
  f'double check that you did all of the behaviours and people correctly, for example that the numbers are all provided.'
)


Here are five potential behaviors that Alice can take in response to the situation:

1. Helping Dorothy move immediately without hesitation.
Positive consequences:
* Helps Dorothy in a time of need (Value: 10, Likelihood: 80)
* Strengthens relationship with grandmother (Value: 9, Likelihood: 70)
* Feels good about herself for helping others (Value: 8, Likelihood: 60)
Negative consequences:
* Tires herself out and affects work performance the next day (Value: -5, Likelihood: 40)
* Has to cancel plans with friends, causing inconvenience (Value: -3, Likelihood: 30)
* Rushes through move and accidentally breaks something valuable (Value: -7, Likelihood: 20)
2. Asking Dorothy if she can help another time.
Positive consequences:
* Allows Alice to prioritize her own needs and schedule (Value: 8, Likelihood: 60)
* Avoids feeling overwhelmed or resentful (Value: 7, Likelihood: 50)
* Shows Dorothy that Alice values their relationship but also has boundaries (Value: 6, Likelihood: 40)
Negative c

In [61]:
import re

def decompose(attitudes):

    attitude_list = re.split(
        r'\n[0-9]\.\s',
        attitudes
    )

    behaviour_list = []

    for attitude in attitude_list[1:]:
        
        behaviours = {}

        items = re.split(
            r'\.\n',
            attitude,
            maxsplit = 1
        )

        behaviours["behaviour"] = items[0]

        items = re.split(
            r'(?:Positive|Negative)\sconsequences:\n',
            items[1]
        )

        items = [item for item in items if item]

        behaviours["consequences"] = []

        for item in items:

            consequences = re.split(
                r'\*\s',
                item
            )
            consequences = [consequence.strip() for consequence in consequences if consequence]

            for consequence in consequences:

                description = re.match(
                    r'(.*)(?=\s\()',
                    consequence
                ).group(1)

                value = int(re.search(
                    r'(?<=Value:\s).*(?=,)',
                    consequence
                ).group(0))

                likelihood = int(re.search(
                    r'(?<=Likelihood:\s).*(?=\))',
                    consequence
                ).group(0))

                behaviours["consequences"].append({
                    "description": description,
                    "value": value,
                    "likelihood": likelihood
                })

        behaviour_list.append(behaviours)
    
    return behaviour_list


behaviour_list = decompose(attitude_ratings)

In [63]:
behaviour_list

[{'behaviour': 'Helping Dorothy move immediately without hesitation',
  'consequences': [{'description': 'Helps Dorothy in a time of need',
    'value': 10,
    'likelihood': 80},
   {'description': 'Strengthens relationship with grandmother',
    'value': 9,
    'likelihood': 70},
   {'description': 'Feels good about herself for helping others',
    'value': 8,
    'likelihood': 60},
   {'description': 'Tires herself out and affects work performance the next day',
    'value': -5,
    'likelihood': 40},
   {'description': 'Has to cancel plans with friends, causing inconvenience',
    'value': -3,
    'likelihood': 30},
   {'description': 'Rushes through move and accidentally breaks something valuable',
    'value': -7,
    'likelihood': 20}]},
 {'behaviour': 'Asking Dorothy if she can help another time',
  'consequences': [{'description': 'Allows Alice to prioritize her own needs and schedule',
    'value': 8,
    'likelihood': 60},
   {'description': 'Avoids feeling overwhelmed or re

In [62]:
normative_ratings = model.sample_text(
  f'Memories of Alice: {behaviors._memory.retrieve_recent(100)}'
  f'Behaviours of Alice: {["\n".join([f"{i+1}. {behaviour_list[i]["behaviour"]}" for i in range(len(behaviour_list))])]}'
  f'Instructions: \n'
  f'For each of the potential behaviours that Alice could take in response to the situation, '
  f'indictate six people or groups of people that would be affected if Alice took that behaviour (do not include Alice). '
  f'For each person or group, provide a rating from -10 to 10 about whether they would approve or disapprove of the behaviour. '
  f'if Alice were to engage in the behaviour. -10 would be the least approval and 10 would be the most approval. '
  f'and from 0 to 100 how much Alice is motivated to take their approval or disapproval into account. '
  f'0 would mean that Alice does not care at all about their approval or disapproval, and 100 would mean that she care the most. '
  f'be sure to each numbers for each ratings and allow for appropriate variation in the ratings'
  f'this should be in the form of (Approval: number, Alice''s Motivation: number) for each potential consequence, '
  f'there should be a motivation number for each of the people for each of the potential behaviours.'
  f'at the end of this, create two lists for each of the approval ratings and motivations for each of the potential consequences. '
  f'double check that you did all of the behaviours and people correctly.'
  f'for example, each of the behaviours should be listed, and each should have a number for approval and motivation '
  f'for each of the people or groups of people.'
  f'provide this information as succinctly as possible. ideally just the behaviour, the people effectec, and the numbers for each '
  f'of the people for each of the behaviours.'
  f'here is an example: \n'
  f'Behaviour 1: \n'
  f'Person 1: Approval: 5, Motivation: 50\n'
  f'Person 2: Approval: 10, Motivation: 100\n'
  f'Person 3: Approval: -5, Motivation: 0\n'
  f'etc.'
  f'the critical thing to to do this consistently for each of the potential behaviours and people.'
)

KeyboardInterrupt: 

In [ ]:
find_ratings = model.sample_text(
    f'given the file below, ' +
    attitude_ratings +
    f'instructions: \n'
    f'create a list of the values and likelihoods for each of the potential consequences. '
    f' for each behaviour found in ' 
    f'Memories of Alice:{behaviors._memory.retrieve_recent(100)}'
    f'provide only the numbers with the values in one list and the likelihoods in another list.'
    f'be sure to do for all behaviours and all potential consequences.'
    f'you should but this into a JSON array with the values and likelihoods as separate lists.'
    f'only the numbers should be in the JSON array.'
)

find_norms = model.sample_text(
    f'given the file below, ' +
    normative_ratings +
    f'instructions: \n'
    f'create a list of the approvals and motivates to comply for each of the behaviours. '
    f' for each behaviour found in ' 
    f'Memories of Alice:{behaviors._memory.retrieve_recent(100)}'
    f'provide only the approvals in one list and the motivations to comply in another list for each behaviour and each of the people under consideration.'
    f'be sure to do for all behaviours and all people that may care.'
    f'you should but this into a JSON array with the approvals and motivations as separate lists.'
    f'only the numbers should be in the JSON array.'
    f'have it be one long JSON array for each of the approvals and motivations rather than separated by behaviour.'
)